# Reliability_testing
## DV
* 因素分析 -> 根據構面取平均 : online_media_pp_mean / voting_mean / offline_media_pp_mean / campaign_worker_pp_mean
* 因素分析 -> 根據構面取綜合得分 : online_media_pp_score / voting_score / offline_media_pp_score / campaign_worker_pp_score
* 先分 online_pp / offline_pp -> 因素分析 -> 根據因素分析結果建立兩個綜合得分 : onlnie_scores / offlnie_scores
## anti_party
* 因素分析 -> 根據構面取平均 : political_polarization_mean / party_image_mean
* 因素分析 -> 根據構面取綜合得分 : political_polarization_score / party_image_score
* 因素分析 -> 全部算一個綜合得分 : anti_party_scores



## Import Package

In [ ]:
! pip install factor_analyzer pingouin

In [ ]:
# for data ETL
import pandas as pd
import ETL
# reliability testing
from factor_analyzer import FactorAnalyzer
import pingouin as pg 

## Load data and Data Cleaning

In [ ]:
raw_data = pd.read_csv('raw_data.csv')

In [ ]:
filter_data = raw_data.loc[raw_data['目前國內政黨當中，請問您是否偏向哪一個政黨？'].isin(['沒有特定支持', '都不支持'])]
filter_data.drop(['您有絶對的權力決定是否要參與本研究。若您願意參與，請務必勾選下列選項：', '請填寫您的電子信箱，以利後續抽獎聯繫使用'], axis=1, inplace=True)

In [ ]:
ml_df = ETL.data_cleaning(filter_data)
ml_df = ml_df.reset_index(drop=True)

In [ ]:
ml_df = ETL.feature_engineering(ml_df)
ml_df = ETL.DV_feature(ml_df)
ml_df = ETL.anti_party_feature(ml_df)
ml_df.shape

In [ ]:
ml_df.columns

## Reliability Testing

## DV testing

### Basic

In [ ]:
online_pp_vars = ['online_pp', 'TV_news_time', 'news_paper_time', 'int_news_time', 'TV_debate',                   'read_media', 'like_media', 'share_media', 'comment_media', 'int_discuss']
offline_pp_vars = ['offline_pp', 'read_election_news', 'read_election_leaflet', 'convince',                        'campaign', 'volunteer', 'election_mayor', 'election_18']
online_pp_tesing_df = ml_df[online_pp_vars]
offline_pp_tesing_df = ml_df[offline_pp_vars]

In [ ]:
alpha_1 = pg.cronbach_alpha(online_pp_tesing_df)
alpha_2 = pg.cronbach_alpha(offline_pp_tesing_df)

print('online_pp Cronbach\'s alpha: ', alpha_1[0])
print('offline_pp Cronbach\'s alpha: ', alpha_2[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0])/2)

### Mean after FAC

In [ ]:
online_media_pp_vars_mean = ['read_media', 'like_media', 'share_media', 'comment_media', 'online_media_pp_mean']
online_media_mean_tesing_df = ml_df[online_media_pp_vars_mean]
voting_vars_mean = ['election_mayor', 'election_18', 'voting_mean']
voting_mean_tesing_df = ml_df[voting_vars_mean]
offline_media_pp_vars_mean = ['read_election_news', 'read_election_leaflet', 'offline_media_pp_mean']
offline_media_mean_tesing_df = ml_df[offline_media_pp_vars_mean]
campaign_worker_pp_vars_mean = ['campaign', 'volunteer', 'campaign_worker_pp_mean']
campaign_worker_mean_tesing_df = ml_df[campaign_worker_pp_vars_mean]

In [ ]:
alpha_1 = pg.cronbach_alpha(online_media_mean_tesing_df)
alpha_2 = pg.cronbach_alpha(voting_mean_tesing_df)
alpha_3 = pg.cronbach_alpha(offline_media_mean_tesing_df)
alpha_4 = pg.cronbach_alpha(campaign_worker_mean_tesing_df)

print('online_media_pp_mean Cronbach\'s alpha: ', alpha_1[0])
print('voting_mean Cronbach\'s alpha: ', alpha_2[0])
print('offline_media_pp_mean Cronbach\'s alpha: ', alpha_3[0])
print('campaign_worker_pp_mean Cronbach\'s alpha: ', alpha_4[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0] + alpha_3[0] + alpha_4[0])/4)

### Total score after FAC

In [ ]:
online_media_pp_vars_score = ['read_media', 'like_media', 'share_media', 'comment_media', 'online_media_pp_score']
online_media_score_tesing_df = ml_df[online_media_pp_vars_score]
voting_vars_score = ['election_mayor', 'election_18', 'voting_score']
voting_score_tesing_df = ml_df[voting_vars_score]
offline_media_pp_vars_score = ['read_election_news', 'read_election_leaflet', 'offline_media_pp_score']
offline_media_score_tesing_df = ml_df[offline_media_pp_vars_score]
campaign_worker_pp_vars_score = ['campaign', 'volunteer', 'campaign_worker_pp_score']
campaign_worker_score_tesing_df = ml_df[campaign_worker_pp_vars_score]

In [ ]:
alpha_1 = pg.cronbach_alpha(online_media_score_tesing_df)
alpha_2 = pg.cronbach_alpha(voting_score_tesing_df)
alpha_3 = pg.cronbach_alpha(offline_media_score_tesing_df)
alpha_4 = pg.cronbach_alpha(campaign_worker_mean_tesing_df)

print('online_media_pp_score Cronbach\'s alpha: ', alpha_1[0])
print('voting_score Cronbach\'s alpha: ', alpha_2[0])
print('offline_media_pp_score Cronbach\'s alpha: ', alpha_3[0])
print('campaign_worker_pp_score Cronbach\'s alpha: ', alpha_4[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0] + alpha_3[0] + alpha_4[0])/4)

### Grouping and FAC get total score

In [ ]:
online_pp_vars = ['onlnie_scores', 'TV_news_time', 'news_paper_time', 'int_news_time', 'TV_debate','read_media', 'like_media', 'share_media', 'comment_media', 'int_discuss']
offline_pp_vars = ['offlnie_scores', 'read_election_news', 'read_election_leaflet', 'convince',                    'campaign', 'volunteer', 'election_mayor', 'election_18']
online_pp_tesing_df = ml_df[online_pp_vars]
offline_pp_tesing_df = ml_df[offline_pp_vars]

In [ ]:
alpha_1 = pg.cronbach_alpha(online_pp_tesing_df)
alpha_2 = pg.cronbach_alpha(offline_pp_tesing_df)

print('onlnie_scores Cronbach\'s alpha: ', alpha_1[0])
print('offlnie_scores Cronbach\'s alpha: ', alpha_2[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0])/2)

## anti_party testing

### Basic

In [ ]:
anti_party_vars = ['anti_party', 'anti_1', 'anti_2', 'anti_3', 'anti_4', 'anti_5']
anti_party_tesing_df = ml_df[anti_party_vars]

In [ ]:
alpha = pg.cronbach_alpha(anti_party_tesing_df)

print('anti_party Cronbach\'s alpha: ', alpha[0])

### Mean after FAC

In [ ]:
political_polarization_mean_vars = ['anti_1', 'political_polarization_mean']
party_image_mean_vars = ['anti_3', 'anti_4', 'anti_5', 'party_image_mean']
political_polarization_mean_tesing_df = ml_df[political_polarization_mean_vars]
party_image_mean_vars_tesing_df = ml_df[party_image_mean_vars]

In [ ]:
alpha_1 = pg.cronbach_alpha(political_polarization_mean_tesing_df)
alpha_2 = pg.cronbach_alpha(party_image_mean_vars_tesing_df)

print('political_polarization_mean Cronbach\'s alpha: ', alpha_1[0])
print('party_image_mean Cronbach\'s alpha: ', alpha_2[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0])/2)

### Total score after FAC two group

In [ ]:
political_polarization_score_vars = ['anti_1', 'political_polarization_score']
party_image_score_vars = ['anti_3', 'anti_4', 'anti_5', 'party_image_score']
political_polarization_score_tesing_df = ml_df[political_polarization_score_vars]
party_image_score_tesing_df = ml_df[party_image_score_vars]

In [ ]:
alpha_1 = pg.cronbach_alpha(political_polarization_score_tesing_df)
alpha_2 = pg.cronbach_alpha(party_image_score_tesing_df)

print('political_polarization_mean Cronbach\'s alpha: ', alpha_1[0])
print('party_image_mean Cronbach\'s alpha: ', alpha_2[0])
print('Total Cronbanc\'s alpha:', (alpha_1[0] + alpha_2[0])/2)

### Total score after FAC

In [ ]:
anti_party_vars = ['anti_party_scores', 'anti_1', 'anti_2', 'anti_3', 'anti_4', 'anti_5']
anti_party_tesing_df = ml_df[anti_party_vars]

In [ ]:
alpha = pg.cronbach_alpha(anti_party_tesing_df)

print('anti_party Cronbach\'s alpha: ', alpha[0])